### Autograd 사용
- 어떤 tensor가 학습에 필요한 tensor라면 `backpropagation`을 통해 gradient를 구해야 함

- `tensor`의 gradient를 구하기 위한 조건

    - `requires_grad = True` 로 설정되어 있어야함, default는 False
    
    - `backpropagation`을 시작하는 지점의 output은 `scalar` 형태   
    
- `tensor`의 gradient를 계산하기 위해서는 output 지점의 tensor에서 `.backward()` 함수를 호출하면 됨

- gradient 값을 확인하려면 `requires_grad=True`로 생성한 tensor에 `.grad` 를 통해 확인할 수 있음

### Autograd 개념

`x`는 2 x 2 크기의 1로 구성된 tensor를 생성한 것이고

`y`는 `requires_grad=True`로 설정하여 역전파 과정을 수행할 수 있게 됨

In [1]:
import torch

x = torch.ones(2, 2)
print(x)

y = torch.ones(2, 2, requires_grad=True)
print(y)

tensor([[1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


앞서 생성한 tensor에 `+3`이라는 연산을 수행했음

연산을 수행한 `y`에 `grad_fn`이 `<AddBackward0>`를 담고 있음

`grad_fn`은 해당 텐서가 어떤 연산을 수행했는지와 관련된 정보를 가지고 있고, 이 정보를 추후에 역전파에 사용함

In [2]:
print(f'x = {x}')

x1 = x + 3
print(f'x1 = {x1}')

print(f'y = {y}')

y1 = y + 3
print(f'y1 = {y1}')

x = tensor([[1., 1.],
        [1., 1.]])
x1 = tensor([[4., 4.],
        [4., 4.]])
y = tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
y1 = tensor([[4., 4.],
        [4., 4.]], grad_fn=<AddBackward0>)


사칙연산을 수행한 후에 역전파에 활용할 정보를 저장하는 `grad_fn`을 보면 각각

`AddBackward0`, `SubBackward0`, `MulBackward0`, `DivBackward0` 등이 저장되어 있음

In [6]:
x = torch.ones(2, 2, requires_grad=True)
print(f'x = {x}')
x1 = x + 2
print(f'x1 = {x1}')
x2 = x - 4
print(f'x2 = {x2}')
x3 = x * 1.2
print(f'x3 = {x3}')
x4 = x / 0.5
print(f'x4 = {x4}')

x = tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
x1 = tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
x2 = tensor([[-3., -3.],
        [-3., -3.]], grad_fn=<SubBackward0>)
x3 = tensor([[1.2000, 1.2000],
        [1.2000, 1.2000]], grad_fn=<MulBackward0>)
x4 = tensor([[2., 2.],
        [2., 2.]], grad_fn=<DivBackward0>)


원래 `reqiures_grad=False`로 default가 되어있는 tensor에 속성을 추가할 때는

`x.requires_grad_(True)` 로 앞으로는 연산 정보를 저장하게 설정해줄 수 있음

하지만, 그 전에 x가 거쳐왔던 연산들은 저장을 하고 있지 않았기 때문에

그 전의 연산들은 없는 상태로 인식됨

In [7]:
x = torch.ones(2, 2)
x1 = x + 3
x2 = x1 * 0.5

print(f'x = {x}')
print(f'x1 = {x1}')
print(f'x2 = {x2}')

x1.requires_grad_(True)
x2.requires_grad_(True)

print(f'x1 = {x1}')
print(f'x2 = {x2}')

x = tensor([[1., 1.],
        [1., 1.]])
x1 = tensor([[4., 4.],
        [4., 4.]])
x2 = tensor([[2., 2.],
        [2., 2.]])
x1 = tensor([[4., 4.],
        [4., 4.]], requires_grad=True)
x2 = tensor([[2., 2.],
        [2., 2.]], requires_grad=True)


#### Derivative

다음과 같은 식에 대해 미분을 한다고 가정.

$f(x) = 9x^4 + 2x^3 + 3x^2 + 6x + 1$

$\cfrac {df(x)}{dx} = \cfrac {d(9x^4 + 2x^3 + 3x^2 + 6x + 1)}{dx} = 36x^3 + 6x^2 + 6x + 6$

만약 $x=2$이면 derivative는 $36 \times 2^3 + 6 \times 2^2 + 6 \times 2 + 6 = 330$

위에서 말했던 대로 backward를 수행하는 y는 스칼라값이여야 함.

In [8]:
x = torch.tensor(2.0, requires_grad=True)
y = 9*x**4 + 2*x**3 + 3*x**2 + 6*x + 1
y.backward()
print(x.grad)

tensor(330.)


Matrix에 대해서 gradient를 구하고 싶다면?

1) 연산 자체에 `.sum()` 또는 `.mean()`을 사용

In [9]:
x = torch.randn(2, 2, requires_grad=True)

y = x + 3
z = (y * y).sum()
z.backward()

print(f'x = {x}')
print(f'y = {y}')
print(f'z = {z}')

print(f'x grad = {x.grad}')
print(f'y grad = {y.grad}')
print(f'z grad = {z.grad}')

x = tensor([[-0.3201,  0.0241],
        [-1.7517, -0.6088]], requires_grad=True)
y = tensor([[2.6799, 3.0241],
        [1.2483, 2.3912]], grad_fn=<AddBackward0>)
z = 23.60271644592285
x grad = tensor([[5.3598, 6.0482],
        [2.4966, 4.7823]])
y grad = None
z grad = None


/home/hwan/anaconda3/envs/torch1.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  if sys.path[0] == '':
/home/hwan/anaconda3/envs/torch1.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull

위에서 볼 수 있다시피 `y, z`에는 실제 grad가 저장되지는 않음

2) 또는 matrix의 `.backward()` 인자를 넣어주면 됨

- `torch.Tensor.backward(gradient=None)`: Computes the gradient of current tensor w.r.t. graph leaves

In [10]:
x = torch.randn(2, 2, requires_grad=True)

y = x + 2

z = (y * y)

y.backward(z)

print(x.grad)

tensor([[8.3303, 0.0103],
        [1.1132, 3.2378]])


#### Partial Derivative

$Q = 3a^3 - b^2$

1) Q는 벡터이므로 `Q.backward()`에 `gradient` 인자를 전달해야한다

2) 또는 Q.sum().backward()와 같이 Q를 scalar값으로 `aggregate`한 뒤 암시적으로 `backward()`를 호출


In [13]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Q = 3*a**3 + 2 * b**2

external_grad = torch.tensor([1., 1.])
Q.backward(external_grad)

print(a.grad, b.grad)

tensor([36., 81.]) tensor([24., 16.])


In [14]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Q = 3*a**3 + 2 * b**2

Q.sum().backward()
print(a.grad, b.grad)

tensor([36., 81.]) tensor([24., 16.])


#### Computational Graph
- 개념적으로 `torch.autograd`는 어떤 tensor에 실행된 모든 연산들의 기록을 `Function` 객체로 구성된 DAG(Directed Acyclic Graph)에 저장한다.

- DAG의 잎(leave)는 입력 tensor 이고, 뿌리(root)는 결과 tensor 임

- Forward에서 `torch.autograd`가 하는 일
    - 요청된 연산을 수행하여 결과 텐서를 계산
    
    - DAG에 연산의 grad_fn을 유지 함

- Backward는 DAG의 root에서 `.backward()`가 호출될 때 시작함, 이 때 `torch.autograd`가 하는 일
    - 각 `.grad_fn` 으로부터 변화도를 계산

    - 각 텐서의 `.grad` 속성에 계산 결과를 쌓는다 (accumulate)

    - chain rule을 사용하여 모든 leaf 텐서들까지 propagate 수행

`torch.autograd`는 `requires_grad`라는 플래그가 `True`인 텐서들에 대해서 연산을 추적함

따라서 derivative 계산이 필요없는 텐서는 속성값을 `False`로 설정해서 DAG에서 제외함

또한 입력 텐서 중 하나라도 `requires_grad=True`이면 결과 텐서도 변화도를 갖게됨

In [16]:
x = torch.randn(5, 5)
y = torch.randn(5, 5)
z = torch.randn(5, 5, requires_grad=True)

a = x + y
b = x + z

print(a.requires_grad)
print(b.requires_grad)

False
True


Finetuning 할 때 모델의 파라미터를 고정 시키기 (파라미터들의 변화도를 필요없게 설정하여 DAG에서 제외시키므로서 `autograd`의 연산량을 줄임)

In [17]:
import torch.nn as nn 
import torch.optim as optim
import torchvision

model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/hwan/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [18]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

Linear Probing을 위해 모델의 마지막 layer만 바꾼다고 가정

In [19]:
model.fc = nn.Linear(512, 10)

optimizer = optim.SGD(model.parameters(), lr=1e-4)

Optimizer가 model의 모든 파라미터를 받지만, derivative를 계산하고, 파라미터를 update할 수 있는 부분은 fc 부분임

#### Training, Eval

`with torch.no_grad():`에서 x가 어떤 연산을 수행했을 때, `requires_grad = False`가 되는 것을 알 수 있음

- `torch.no_grad()` -- Context-manager that disabled gradient calculations
    - `torch.autograd`의 엔진이 꺼져버린다. = 앞으로 gradient의 자동 tracking을 하지 않겠다
    
    - 어차피 `loss.backward()`를 호출 안하고, backpropagation도 안하는데 굳이 gradient 계산을 안하게 하는 이유?

    - 메모리 사용량을 줄이고 연산 속도를 높히려고

- `model.eval()` -- Batch normalization, dropout과 같은 것들을 evaluation mode로 바꿔줌

In [22]:
x = torch.tensor(1.0, requires_grad=True)

print(x.requires_grad)

with torch.no_grad():
    print(x.requires_grad)
    print((x + 3).requires_grad)

print(x.requires_grad)

True
True
False
True


### 여러가지 유용한 함수들
- `torch.gather(input, dim, index)` : Gathers values along an axis specified by `dim`

- `torch.tensor.gather(dim, index)`

- `torch.tensor.expand(sizes)` : Returns a new view of the `self` tensor with singleton dimensions expanded to a larger size

- `torch.tensor.repeat(sizes)` : Repeats this tensor along the specified dimensions / Unlike `expand()`, this function copies the tensor's data

- `torch.topk(input, k, dim=None, largest=True, sorted=True)` : Returns the k largest elements of the given `input` tensor along a given dimension